In [9]:
from config import api_key
import requests
import json
import pandas as pd
from pprint import pprint
import time
from splinter import browser
from bs4 import BeautifulSoup

In [12]:
url = "https://www.imdb.com/search/title/?year=2019&title_type=feature&"

# url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
#browser.visit(url)

In [15]:


count = 0

i = 1

movie_list = []

while count < 11470:
    
    if i == 1:
        url = "https://www.imdb.com/search/title/?year=2019&title_type=feature&"
        page= requests.get(url)
        soup= BeautifulSoup(page.text, "html.parser")
        raw_html = soup.find("tbody", {"class": "lister-list"}).findAll("tr")
    
        
        for html in raw_html:
            title = html.find("td", {"class": "titleColumn"}).find("a").get_text()
            year = html.find("td", {"class": "titleColumn"}).find("span").getText()[1:5]
            rating= html.find("td", {"class": "ratingColumn imdbRating"}).find("strong").get_text()
            rating= float(rating)

            raw_list= [title, year, rating]
            movie_list.append(raw_list)
            count += 1
   
        if count%50 == 0:
            i = i + 50
            url2 = f"https://www.imdb.com/search/title/?title_type=feature&year=2019-01-01,2019-12-31&start={i}&ref_=adv_nxt"

    else:
        
        
        page= requests.get(url2)
        soup= BeautifulSoup(page.text, "html.parser")
        raw_html = soup.find("tbody", {"class": "lister-list"}).findAll("tr")
    
        
        for html in raw_html:
            title = html.find("td", {"class": "titleColumn"}).find("a").get_text()
            year = html.find("td", {"class": "titleColumn"}).find("span").getText()[1:5]
            rating= html.find("td", {"class": "ratingColumn imdbRating"}).find("strong").get_text()
            rating= float(rating)

            raw_list= [title, year, rating]
            movie_list.append(raw_list)
            count += 1
        
        
        if count%50 == 0:
            i = i + 50
            url2 = f"https://www.imdb.com/search/title/?title_type=feature&year=2019-01-01,2019-12-31&start={i}&ref_=adv_nxt"
            
            
            

AttributeError: 'NoneType' object has no attribute 'findAll'

In [40]:
from bs4 import BeautifulSoup

In [41]:
url="https://www.imdb.com/search/title/?year=2019&title_type=feature&"

In [42]:
page= requests.get(url)

In [43]:
page

<Response [200]>

In [45]:
soup= BeautifulSoup(page.text, "html.parser")

In [46]:
raw_html = soup.find("tbody", {"class": "lister-list"}).findAll("tr")

In [48]:

movie_list = []
for html in raw_html:
    title = html.find("td", {"class": "titleColumn"}).find("a").get_text()
    year = html.find("td", {"class": "titleColumn"}).find("span").getText()[1:5]
    rating= html.find("td", {"class": "ratingColumn imdbRating"}).find("strong").get_text()
    rating= float(rating)
        
    raw_list= [title, year, rating]
    movie_list.append(raw_list)

In [49]:
import pandas as pd

In [50]:
thedf = pd.DataFrame(movie_list)

In [60]:
imdb_df = thedf.rename(columns={0:"movie_name", 1:"year", 2:"rating"})

In [61]:
movie_name = imdb_df['movie_name']

In [64]:
imdb_df.set_index('movie_name', inplace=True)

In [65]:
imdb_df

,year,rating
movie_name,,
The Shawshank Redemption,1994,9.2
The Godfather,1972,9.1
The Godfather: Part II,1974,9.0
The Dark Knight,2008,9.0
12 Angry Men,1957,8.9
...,...,...
La battaglia di Algeri,1966,8.0
"Swades: We, the People",2004,8.0
Hera Pheri,2000,8.0


In [66]:
print(movie_name)

0                               The Shawshank Redemption
1                                          The Godfather
2                                 The Godfather: Part II
3                                        The Dark Knight
4                                           12 Angry Men
                             ...                        
245                               La battaglia di Algeri
246                               Swades: We, the People
247                                           Hera Pheri
248                                          Kumonosu-jô
249    Shin seiki Evangelion Gekijô-ban: Air/Magokoro...
Name: movie_name, Length: 250, dtype: object


In [ ]:
# NYT api

In [ ]:
url = 'https://api.nytimes.com/svc/movies/v2/reviews/search.json?'
response_list = []
alternate_list = []
movies_nyt = []
movies_wedidntget = []
for x in movie_name:
    try:
        base_url = url + "query=" + str(x) + "&api-key=" + api_key
        response = requests.get(base_url).json()
        response_list.append(response)
        if response['results'] != []:
            alternate_list.append(response['results'][int(0)])
            movies_nyt.append(x)
            time.sleep(6)
    except:
        movies_wedidntget.append(x)
        next
        time.sleep(6)

In [ ]:
alternate_list

In [ ]:
df = pd.read_json('alternate_list')
df

In [ ]:
# with open('alternate.json', 'w', encoding='UTF-8') as hey:
#     json.dump(alternate_list, hey, ensure_ascii=False, indent=4)

In [ ]:
thedf = pd.read_json('alternate.json')
thedf

In [27]:
from rotten_tomatoes_client import RottenTomatoesClient

meterclass = []
meterscore = []
movies_weget = []
movies_wedontget = []
for x in movie_names:
    try:
        result = RottenTomatoesClient.search(term=x, limit=1)
        if result['movies'][int(0)]['meterClass'] and result['movies'][int(0)]['meterScore']:
            meterclass.append(result['movies'][int(0)]['meterClass'])
            meterscore.append(result['movies'][int(0)]['meterScore'])
            movies_weget.append(x)
            print(x)
    except:
        movies_wedontget.append(x)
        next

The Shawshank Redemption
The Godfather
The Godfather: Part II
The Dark Knight
12 Angry Men
Schindler's List
The Lord of the Rings: The Return of the King
Pulp Fiction
The Lord of the Rings: The Fellowship of the Ring
Fight Club
Forrest Gump
Inception
Star Wars: Episode V - The Empire Strikes Back
The Lord of the Rings: The Two Towers
The Matrix
Goodfellas
One Flew Over the Cuckoo's Nest
Shichinin no samurai
Se7en
La vita ? bella
Cidade de Deus
The Silence of the Lambs
It's a Wonderful Life
Star Wars
Saving Private Ryan
Hamilton
The Green Mile
Gisaengchung
Interstellar
L?on
The Usual Suspects
The Lion King
Back to the Future
The Pianist
Terminator 2: Judgment Day
American History X
Modern Times
Gladiator
City Lights
The Departed
The Intouchables
Whiplash
The Prestige
Hotaru no haka
Once Upon a Time in the West
Casablanca
Nuovo Cinema Paradiso
Rear Window
Apocalypse Now
Memento
Raiders of the Lost Ark
The Great Dictator
Joker
The Lives of Others
Django Unchained
Paths of Glory
The Shinin

In [31]:
rot_df = pd.DataFrame({
    "movie_name":movies_weget,
    "meter_class" : meterclass,
    "meter_score" : meterscore
})

In [35]:
rot_df.set_index('movie_name', inplace=True)

KeyError: "None of ['movie_name'] are in the columns"

In [37]:
rot_df

,meter_class,meter_score
movie_name,,
The Shawshank Redemption,certified_fresh,90
The Godfather,certified_fresh,98
The Godfather: Part II,certified_fresh,98
The Dark Knight,certified_fresh,87
12 Angry Men,certified_fresh,100
...,...,...
Drishyam,fresh,80
The Help,certified_fresh,76
The Princess Bride,certified_fresh,97
